In [49]:
#Librerias necesarias, en esta version, 1.6, no existe el SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext

In [29]:
#Inicializar el SparkContext
sc = SparkContext.getOrCreate()

In [41]:
#Inicializar el Hive Context
sqlContext = HiveContext(sc)

<span style='color:Blue'>Creacion de DF por cada tabla en Hive</span>

In [51]:
vuelos = sqlContext.sql("SELECT * FROM ejercicio1.vuelos")

In [56]:
retrasos = sqlContext.sql("SELECT * FROM ejercicio1.retrasos")

In [128]:
fecha = sqlContext.sql("SELECT * FROM ejercicio1.fecha")

In [178]:
paises = sqlContext.sql("SELECT * FROM ejercicio1.paises")

<span style='color:Blue'>3.Crear una nueva tabla en Hive, que contenga la siguiente información: Vuelo, Origen, Destino, Retraso.</span>

In [201]:
#Inner join de tablas Vuelos y Retrasos
HD_ResumenVuelos = sqlContext.sql("SELECT v.Vuelo, v.Origen, v.Destino, r.Retraso FROM ejercicio1.vuelos v INNER JOIN ejercicio1.retrasos r ON (v.vuelo=r.vuelo)")
HD_ResumenVuelos.show(5)

+-----+------+-------+-------+
|Vuelo|Origen|Destino|Retraso|
+-----+------+-------+-------+
| 6005|   MEX|    ESP|     52|
| 2021|   COL|    MEX|     27|
| 5528|   ARG|    COL|     36|
| 7965|   BRA|    ARG|     82|
| 8717|   USA|    BRA|     29|
+-----+------+-------+-------+
only showing top 5 rows



In [200]:
#Crear tabla HD_ResumenVuelos en la ruta hdfs del ejercicio
HD_ResumenVuelos.write.option("path", "hdfs:/home/training/Desktop/Fuentes/HD_ResumenVuelos").saveAsTable("ejercicio1.HD_ResumenVuelos")

<span style='color:Blue'>(A) ¿De qué país salieron más aviones?</span>

<span style='color: Green'>(A) SQL</span>

In [183]:
#Left join de Vuelos con Paises, con key orign y cod_pais, agrupado por pais y ordenado desc.
HD_ResumenVuelosSalidasSQL = sqlContext.sql("SELECT p.Pais, COUNT(v.vuelo) As Salidas FROM ejercicio1.vuelos v LEFT JOIN ejercicio1.paises p ON (v.origen=p.cod_pais) GROUP BY p.Pais ORDER BY Salidas DESC")
HD_ResumenVuelosSalidasSQL.show(3)
#De España(8) salieron mas vuelos

+--------------+-------+
|          Pais|Salidas|
+--------------+-------+
|        España|      8|
|       Francia|      7|
|Estados Unidos|      6|
+--------------+-------+
only showing top 3 rows



<span style='color: Green'>(A) PySpark</span>

In [184]:
#Crear DF HD_ResumenVuelosSalidas
HD_ResumenVuelosSalidas = vuelos.alias("v").join(paises.alias("p"), vuelos.origen == paises.cod_pais)
HD_ResumenVuelosSalidas = HD_ResumenVuelosSalidas.select("p.Pais", "v.Vuelo")
HD_ResumenVuelosSalidas.show(5)

+--------------+-----+
|          Pais|Vuelo|
+--------------+-----+
|        Mexico| 6005|
|      Colombia| 2021|
|     Argentina| 5528|
|        Brasil| 7965|
|Estados Unidos| 8717|
+--------------+-----+
only showing top 5 rows



In [185]:
#Left join de Vuelos con Paises, con key orign y cod_pais, agrupado por pais y ordenado desc.
groupVuelos = HD_ResumenVuelosSalidas.groupBy("Pais")
groupVuelos = groupVuelos.agg({'Vuelo':'count'})
groupVuelos = groupVuelos.withColumnRenamed("count(Vuelo)", "Salidas")
groupVuelos = groupVuelos.orderBy("Salidas", ascending=False)
groupVuelos.show(3)
#De España(8) salieron mas vuelos

+--------------+-------+
|          Pais|Salidas|
+--------------+-------+
|        España|      8|
|       Francia|      7|
|Estados Unidos|      6|
+--------------+-------+
only showing top 3 rows



In [186]:
#Crear tabla HM_ResumenDias en la ruta hdfs del ejercicio
groupVuelos.write.option("path", "hdfs:/home/training/Desktop/Fuentes/HD_ResumenVuelosSalidas").saveAsTable("ejercicio1.HD_ResumenVuelosSalidas")

<span style='color:Blue'>(B)¿A qué país llegaron más aviones?</span>

<span style='color: Green'>(B) SQL</span>

In [188]:
#Left join de Vuelos con Paises, con key destino y cod_pais, agrupado por pais y ordenado desc.
HD_ResumenVuelosLlegadasSQL = sqlContext.sql("SELECT p.Pais, COUNT(v.vuelo) As Llegadas FROM ejercicio1.vuelos v LEFT JOIN ejercicio1.paises p ON (v.destino=p.cod_pais) GROUP BY p.Pais ORDER BY Llegadas DESC")
HD_ResumenVuelosLlegadasSQL.show(3)
#A Mexico(7) llegaron mas vuelos

+------+--------+
|  Pais|Llegadas|
+------+--------+
|Mexico|       7|
| Rusia|       6|
|  Peru|       6|
+------+--------+
only showing top 3 rows



In [189]:
#Crear DF HD_ResumenVuelosLlegadas
HD_ResumenVuelosLlegadas = vuelos.alias("v").join(paises.alias("p"), vuelos.destino == paises.cod_pais)
HD_ResumenVuelosLlegadas = HD_ResumenVuelosLlegadas.select("p.Pais", "v.Vuelo")
HD_ResumenVuelosLlegadas.show(5)

+---------+-----+
|     Pais|Vuelo|
+---------+-----+
|   España| 6005|
|   Mexico| 2021|
| Colombia| 5528|
|Argentina| 7965|
|   Brasil| 8717|
+---------+-----+
only showing top 5 rows



<span style='color: Green'>(B) PySpark</span>

In [190]:
#Join de tablas Vuelos con Retrasos, grop by Destino, con key vuelo y ordenado ascendentemente
groupVuelos = HD_ResumenVuelosLlegadas.groupBy("Pais")
groupVuelos = groupVuelos.agg({'Vuelo':'count'})
groupVuelos = groupVuelos.withColumnRenamed("count(Vuelo)", "Llegadas")
groupVuelos = groupVuelos.orderBy("Llegadas", ascending=False)
groupVuelos.show(3)
#A Mexico(7) llegaron mas vuelos

+------+--------+
|  Pais|Llegadas|
+------+--------+
|Mexico|       7|
| Rusia|       6|
|  Peru|       6|
+------+--------+
only showing top 3 rows



In [191]:
#Crear tabla HD_ResumenVuelosLlegadas en la ruta hdfs del ejercicio
groupVuelos.write.option("path", "hdfs:/home/training/Desktop/Fuentes/HD_ResumenVuelosLlegadas").saveAsTable("ejercicio1.HD_ResumenVuelosLlegadas")

<span style='color:Blue'>(C)¿Qué día hubo más vuelos? ¿Y menos?</span>

<span style='color: Green'>(C) SQL</span>

In [192]:
#Select de tabla Fecha, agroupado por dia, contando los vuelos 
HD_ResumenDiasSQL = sqlContext.sql("SELECT Dia, Count(Vuelo) as Vuelos FROM ejercicio1.fecha GROUP BY Dia ORDER BY Vuelos DESC")
HD_ResumenDiasSQL.show(5)
#Dias 1, 9 y 20 tuvieron mas vuelos (7)

HD_ResumenDiasSQLMenor = sqlContext.sql("SELECT Dia, Count(Vuelo) as Vuelos FROM ejercicio1.fecha GROUP BY Dia ORDER BY Vuelos ASC")
#HM_ResumenDiasSQLMenor.show(5)
#Dias 23, 11, 17, 21 y 8 tuvieron menos vuelos (1)

+---+------+
|Dia|Vuelos|
+---+------+
|  1|     7|
|  9|     7|
| 20|     7|
|  5|     6|
| 18|     6|
+---+------+
only showing top 5 rows



<span style='color: Green'>(C) PySpark</span>

In [193]:
#Select de tabla Fecha, agroupado por dia, contando los vuelos 
HD_ResumenDias = fecha.groupBy("Dia")
HD_ResumenDias = HD_ResumenDias.agg({'Vuelo':'count'})
HD_ResumenDias = HD_ResumenDias.withColumnRenamed("count(Vuelo)", "Vuelos")
HD_ResumenDias = HD_ResumenDias.orderBy("Vuelos", ascending=False)
HD_ResumenDias.show(5)
#Dias 1, 9 y 20 tuvieron mas vuelos (7)

HD_ResumenDiasMenor = HD_ResumenDias.orderBy("Vuelos", ascending=True)
#HM_ResumenDiasMenor.show(5)
#Dias 23, 11, 17, 21 y 8 tuvieron menos vuelos (1)

+---+------+
|Dia|Vuelos|
+---+------+
|  1|     7|
|  9|     7|
| 20|     7|
|  5|     6|
| 18|     6|
+---+------+
only showing top 5 rows



In [194]:
#Crear tabla HD_ResumenDias en la ruta hdfs del ejercicio
HD_ResumenDias.write.option("path", "hdfs:/home/training/Desktop/Fuentes/HD_ResumenDias").saveAsTable("ejercicio1.HD_ResumenDias")

<span style='color:Blue'>(D)¿Qué día hubo más retrasos? ¿Y menos?</span>

<span style='color: Green'>(D) SQL</span>

In [167]:
#Join de tabla Fecha on Retrasos con key vuelo, agrupar por dia y ordenar de forma asc/desc
HD_ResumenRetrasosSQL = sqlContext.sql("SELECT f.Dia, Count(r.retraso) As Retrasos FROM ejercicio1.fecha f INNER JOIN ejercicio1.retrasos r ON (f.vuelo=r.vuelo) GROUP BY f.dia ORDER BY Retrasos DESC")
#El dia 20 de Febrero tuvo mas retrasos

HD_ResumenRetrasosMenorSQL = sqlContext.sql("SELECT f.Dia, Count(r.retraso) As Retrasos FROM ejercicio1.fecha f INNER JOIN ejercicio1.retrasos r ON (f.vuelo=r.vuelo) GROUP BY f.dia ORDER BY Retrasos ASC")
#Los dias 23, 21, 17 y 11 de Febrero tuvo menos retrasos 

In [168]:
HD_ResumenRetrasosSQL.show(5)

+---+--------+
|Dia|Retrasos|
+---+--------+
| 20|       8|
|  1|       7|
|  9|       7|
|  5|       6|
| 18|       6|
+---+--------+
only showing top 5 rows



<span style='color: Green'>(D) PySpark</span>

In [169]:
#Crear tabla HD_ResumenRetrasos que contenga los min de retraso por dia 
HD_ResumenRetrasos = fecha.alias("f").join(retrasos.alias("r"), fecha.vuelo == retrasos.vuelo)
HD_ResumenRetrasos = HD_ResumenRetrasos.select("f.Dia","r.Retraso")
HD_ResumenRetrasos.show(3)

+---+-------+
|Dia|Retraso|
+---+-------+
|  2|     52|
| 25|     27|
| 18|     36|
+---+-------+
only showing top 3 rows



In [171]:
#Join de tablas Fecha con Retrasos, group by Dia, con key vuelo y ordenado ascendentemente
groupRetrasos = HD_ResumenRetrasos.groupBy("Dia")
groupRetrasos = groupRetrasos.agg({'Retraso':'count'})
groupRetrasos = groupRetrasos.withColumnRenamed("count(Retraso)", "Retrasos")
groupRetrasos = groupRetrasos.orderBy("Retrasos", ascending=False)
groupRetrasos.show(5)
#El dia 20 de Febrero tuvo mas retrasos 

HD_ResumenRetrasosMenor = groupRetrasos.orderBy("Retrasos", ascending=True)
#HD_ResumenRetrasosMenor.show(5)
#Los dias 23, 21, 17 y 11 de Febrero tuvieron menos retrasos 

+---+--------+
|Dia|Retrasos|
+---+--------+
| 20|       8|
|  1|       7|
|  9|       7|
|  5|       6|
| 18|       6|
+---+--------+
only showing top 5 rows



In [172]:
#Crear tabla HD_ResumenRetrasos en la ruta hdfs del ejercicio
groupRetrasos.write.option("path", "hdfs:/home/training/Desktop/Fuentes/HD_ResumenRetrasos").saveAsTable("ejercicio1.HD_ResumenRetrasos")